In [1]:
import pandas as pd
import numpy as np
import win32com.client
import re

#엑셀을 열기 위한 모듈

excel = win32com.client.Dispatch("Excel.Application")
excel.Visible = True



#엑셀의 column 이름을 str 타입으로 바꿔주는 함수

def column_naming(df):
    str = input()
    
    str_list = str.split()
    df.columns = str_list
    return df

#엑셀의 시간 데이터를 datetime 객체로 바꿔주는 함수

def transform_datetype(df):
    df['날짜'] = df['날짜'].astype('str')
    df['날짜'] = pd.to_datetime(df['날짜'])
    df.set_index(df['날짜'], inplace=True)
    df = df.drop('날짜', 1)
    return df

#데이터의 발생빈도비율을 구하는 함수

def add_freq(df):
    for i in df.columns:
        if i == 'gatheredCount':
            continue
    
        else:
            try:
                df[i+' 언급 비율']=df[i]/df['gatheredCount']
                
            except:
                pass
    
    return df

#데이터의 발생빈도합계/비율을 월, 분기, 반기, 연으로 나눠서 리스트에 저장하는 함수

def conv_df(df):
    freq = ['M', 'Q', '2Q', 'Y']
    res = {}
    for i in freq: 
        res[i] = i        
        
    for j in freq:
        tr_1 = df.resample(rule=j).sum()
        tr_2 = df.resample(rule=j).mean()

        for i in tr_1.columns:
            if i == 'gatheredCount':
                continue
            if i.endswith(' 언급 비율'):
                tr_1.drop(i, axis=1, inplace = True)
            else:
                tr_1.rename(columns = {i : i+' 언급량'}, inplace = True)

        for i in tr_2.columns:
            if i == 'gatheredCount':
                tr_2.drop(i, axis=1, inplace = True)
                continue
            else:
                tr_2.rename(columns = {i : i+' 평균값'}, inplace = True)
        output = pd.concat([tr_1,tr_2],axis=1, verify_integrity=False)
        cols = output.columns.tolist()
        
        
        num = cols.index('gatheredCount')
        cols = cols[num+1:]+  cols[:num+1]
        print(cols)
        output = output[cols]
        res[j] = output
        
        
    return res

#각 시간 단위별 표시형식을 변경하는 함수          
def trans_korean(input_dict):
    result = input_dict
    for key in result.keys():
        value = result[key]
        if key == 'M':
            value.reset_index(inplace=True)
            value['날짜'] = value['날짜'].astype('str')
            value['날짜'] = pd.to_datetime(value['날짜'])
            value['날짜'] = value['날짜'].dt.year.astype('str') +'년 '+ value['날짜'].dt.month.astype('str') + '월'
            value.set_index(value['날짜'], inplace=True)
            value = value.drop('날짜', 1)
            result[key] = value
            
        elif key == 'Q':
            value.reset_index(inplace=True)
            value['날짜'] = value['날짜'].astype('str')
            value['날짜'] = pd.to_datetime(value['날짜'])
            value['날짜'] = pd.DatetimeIndex(value['날짜']).year
            value['날짜'] = value['날짜'].astype('str')
            for i in range(value.shape[0]):
                value.loc[i, '날짜'] = value['날짜'][i] + ' ' + str((i)%4 + 1) +'Q'
            value.set_index(value['날짜'], inplace=True)
            value = value.drop('날짜', 1)
            result[key] = value
            
        elif key == '2Q':
            value.reset_index(inplace=True)
            value['날짜'] = value['날짜'].astype('str')
            value['날짜'] = pd.to_datetime(value['날짜'])
            value['날짜'] = pd.DatetimeIndex(value['날짜']).year
            value['날짜'] = value['날짜'].astype('str')
            for i in range(value.shape[0]):
                value.loc[i, '날짜'] = value['날짜'][i] + ' ' + str((i)%2 + 1) +'H'
            value.set_index(value['날짜'], inplace=True)
            value = value.drop('날짜', 1)
            result[key] = value
            
        else:
            value.reset_index(inplace=True)
            value['날짜'] = value['날짜'].astype('str')
            value['날짜'] = pd.to_datetime(value['날짜'])
            value['날짜'] = pd.DatetimeIndex(value['날짜']).year
            value['날짜'] = value['날짜'].astype('str')
            for i in range(value.shape[0]):
                value.loc[i, '날짜'] = value['날짜'][i]
            value.set_index(value['날짜'], inplace=True)
            value = value.drop('날짜', 1)
            result[key] = value
            
    return result

#Dataframe을 멤버로 보유하는 리스트를 입력받아, 각 Dataframe을 엑셀의 시트로 저장하는 함수

def export_excel(input_dict):
    title = input("저장할 파일명을 입력하세요. : ")
    title = title+'.xlsx'
    period_list = list(input_dict.keys())
    
    with pd.ExcelWriter(title) as writer:
        for i in period_list:
            input_dict[i].to_excel(writer, sheet_name=i)
    
    return title

#엑셀을 열기 위한 함수
def excel_open(input_filename):
    input_filename = 'C:\\Users\\rlawh\\'+input_filename
    wb = excel.Workbooks.Open(input_filename)

file_name = input("불러올 파일명을 입력하세요: ")
df = pd.read_csv(file_name+".csv")
df = transform_datetype(df)
df = add_freq(df)
result = conv_df(df)
result = trans_korean(result)
title = export_excel(result)
excel_open(title)
title



불러올 파일명을 입력하세요: pr2
['대깨맥 평균값', '슼갈 평균값', '대깨맥 언급 비율 평균값', '슼갈 언급 비율 평균값', '대깨맥 언급량', '슼갈 언급량', 'gatheredCount']
['대깨맥 평균값', '슼갈 평균값', '대깨맥 언급 비율 평균값', '슼갈 언급 비율 평균값', '대깨맥 언급량', '슼갈 언급량', 'gatheredCount']
['대깨맥 평균값', '슼갈 평균값', '대깨맥 언급 비율 평균값', '슼갈 언급 비율 평균값', '대깨맥 언급량', '슼갈 언급량', 'gatheredCount']
['대깨맥 평균값', '슼갈 평균값', '대깨맥 언급 비율 평균값', '슼갈 언급 비율 평균값', '대깨맥 언급량', '슼갈 언급량', 'gatheredCount']
저장할 파일명을 입력하세요. : asd


'asd.xlsx'